Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install numerapi
!pip install vecstack;

In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-07-18 10:08:13,394 INFO numerapi.utils: target file already exists
2021-07-18 10:08:13,395 INFO numerapi.utils: download complete
2021-07-18 10:08:13,397 INFO numerapi.base_api: unzipping file...


In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 10s, sys: 18.5 s, total: 1min 28s
Wall time: 1min 30s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
%%time

model_e = LGBMRegressor(random_state=rand)
gc.collect()

model = LGBMRegressor(random_state=rand)
model.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

y_pred_model = model.predict(tournament_data[feature_names])
gc.collect()

tournament_data[PREDICTION_NAME] = y_pred_model


CPU times: user 3min 18s, sys: 5.63 s, total: 3min 23s
Wall time: 19.5 s


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488480
n000920ed083903f    0.487922
n0038e640522c4a6    0.516811
n004ac94a87dc54b    0.499933
n0052fe97ea0c05f    0.502267
                      ...   
nffd874d091bc7dd    0.533388
nffdbdf8cab43009    0.499499
nffe56d85bb8a863    0.501996
nffe6104a95075f9    0.492521
nfff57be2f397aeb    0.489509
Name: prediction, Length: 1768754, dtype: float64

In [ ]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


In [ ]:
%%time
tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

CPU times: user 8min 30s, sys: 8min 47s, total: 17min 17s
Wall time: 58.7 s


In [ ]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.549566
n000920ed083903f    0.497427
n0038e640522c4a6    0.532135
n004ac94a87dc54b    0.572584
n0052fe97ea0c05f    0.565850
                      ...   
nffd874d091bc7dd    0.710163
nffdbdf8cab43009    0.544683
nffe56d85bb8a863    0.558303
nffe6104a95075f9    0.517121
nfff57be2f397aeb    0.530609
Name: prediction, Length: 1768754, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol_1.csv')

!cp FisaGol_1.csv "drive/My Drive/PHOENIXSIGMA/"